Link: http://saifmohammad.com/WebPages/EmotionIntensity-SharedTask.html


In [75]:
from __future__ import print_function, division
import numpy as np
import pandas as pd
import re

In [37]:
# train data
col_names = ['id', 'expression', 'emotion', 'score']
anger_train = pd.read_csv('http://saifmohammad.com/WebDocs/EmoInt%20Train%20Data/anger-ratings-0to1.train.txt', 
                          sep='\t', header = None, names = col_names)
fear_train = pd.read_csv('http://saifmohammad.com/WebDocs/EmoInt%20Train%20Data/fear-ratings-0to1.train.txt', 
                          sep='\t', header = None, names = col_names)
joy_train = pd.read_csv('http://saifmohammad.com/WebDocs/EmoInt%20Train%20Data/joy-ratings-0to1.train.txt', 
                          sep='\t', header = None, names = col_names)
sadness_train = pd.read_csv('http://saifmohammad.com/WebDocs/EmoInt%20Train%20Data/sadness-ratings-0to1.train.txt', 
                          sep='\t', header = None, names = col_names)
train = pd.concat([anger_train, fear_train, joy_train, sadness_train])

,id,expression,emotion,score
0,10000,How the fu*k! Who the heck! moved my fridge!.....,anger,0.938
1,10001,So my Indian Uber driver just called someone t...,anger,0.896
2,10002,@DPD_UK I asked for my parcel to be delivered ...,anger,0.896
3,10003,so ef whichever butt wipe pulled the fire alar...,anger,0.896
4,10004,Don't join @BTCare they put the phone down on ...,anger,0.896


In [40]:
# test data
anger_test = pd.read_csv('http://saifmohammad.com/WebDocs/EmoInt%20Test%20Gold%20Data/anger-ratings-0to1.test.gold.txt', 
                          sep='\t', header = None, names = col_names)
fear_test = pd.read_csv('http://saifmohammad.com/WebDocs/EmoInt%20Test%20Gold%20Data/fear-ratings-0to1.test.gold.txt', 
                          sep='\t', header = None, names = col_names)
joy_test = pd.read_csv('http://saifmohammad.com/WebDocs/EmoInt%20Test%20Gold%20Data/joy-ratings-0to1.test.gold.txt', 
                          sep='\t', header = None, names = col_names)
sadness_test = pd.read_csv('http://saifmohammad.com/WebDocs/EmoInt%20Test%20Gold%20Data/sadness-ratings-0to1.test.gold.txt', 
                          sep='\t', header = None, names = col_names)
test = pd.concat([anger_test, fear_test, joy_test, sadness_test])

In [32]:
def cleaning(expression):
    #removing @, #
    expression=re.sub(r'@\w+', '<subject>', expression)
    expression=re.sub(r'#', '', expression)
    
    #lower case
    expression=expression.lower()
    return expression
    

In [42]:
train.expression = train.expression.apply(cleaning)
test.expression = test.expression.apply(cleaning)

In [43]:
train.head()

,id,expression,emotion,score
0,10000,how the fu*k! who the heck! moved my fridge!.....,anger,0.938
1,10001,so my indian uber driver just called someone t...,anger,0.896
2,10002,<subject> i asked for my parcel to be delivere...,anger,0.896
3,10003,so ef whichever butt wipe pulled the fire alar...,anger,0.896
4,10004,don't join <subject> they put the phone down o...,anger,0.896


In [44]:
test.head()

,id,expression,emotion,score
0,10941,at the point today where if someone says somet...,anger,0.319
1,10942,<subject> it's game day!!!! t minus 14:3...,anger,0.144
2,10943,this game has pissed me off more than any othe...,anger,0.898
3,10944,<subject> i've just found out it's candice and...,anger,0.271
4,10945,<subject> <subject> <subject> <subject> if he ...,anger,0.646


In [47]:
len(train), len(test)

(3613, 3142)

## Simple classification 
### Logistic regression

In [49]:
sentences = list(train.expression) + list(test.expression)

In [99]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=0, lowercase=True)
vectorizer.fit(sentences)
vectorizer.vocabulary_

{'how': 5114,
 'the': 10812,
 'fu': 4222,
 'who': 11875,
 'heck': 4893,
 'moved': 6953,
 'my': 7033,
 'fridge': 4179,
 'should': 9724,
 'knock': 5933,
 'landlord': 6005,
 'door': 3176,
 'angry': 639,
 'mad': 6409,
 'so': 9968,
 'indian': 5359,
 'uber': 11281,
 'driver': 3244,
 'just': 5792,
 'called': 1712,
 'someone': 10020,
 'word': 11999,
 'if': 5235,
 'wasn': 11711,
 'in': 5318,
 'moving': 6961,
 'vehicle': 11511,
 'have': 4838,
 'jumped': 5777,
 'out': 7799,
 'disgusted': 3063,
 'subject': 10398,
 'asked': 822,
 'for': 4087,
 'parcel': 7909,
 'to': 10994,
 'be': 1131,
 'delivered': 2845,
 'pick': 8124,
 'up': 11417,
 'store': 10315,
 'not': 7441,
 'address': 385,
 'fuming': 4253,
 'poorcustomerservice': 8295,
 'ef': 3369,
 'whichever': 11853,
 'butt': 1672,
 'wipe': 11943,
 'pulled': 8622,
 'fire': 3957,
 'alarm': 521,
 'davis': 2740,
 'bc': 1125,
 'was': 11705,
 'sound': 10058,
 'asleep': 826,
 'pissed': 8166,
 'upset': 11432,
 'tired': 10977,
 'sad': 9332,
 'hangry': 4772,
 'don

In [100]:
X_train = vectorizer.transform(list(train.expression)).toarray()
X_test = vectorizer.transform(list(test.expression)).toarray()

In [101]:
X_train.shape, X_test.shape

((3613, 12249), (3142, 12249))

In [143]:
# encoding
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
emotion_encoder = LabelEncoder()
y_train = emotion_encoder.fit_transform(train.emotion)
y_test = emotion_encoder.fit_transform(test.emotion)

emotion_one_hot_encoder = OneHotEncoder()
y_train = emotion_one_hot_encoder.fit_transform(y_train.reshape((-1, 1)))
y_test = emotion_one_hot_encoder.fit_transform(y_test.reshape((-1, 1)))

In [144]:
y_train

<3613x4 sparse matrix of type '<class 'numpy.float64'>'
	with 3613 stored elements in Compressed Sparse Row format>

In [106]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", score)

Accuracy: 0.8058561425843411


In [123]:
emotion_encoder.inverse_transform(classifier.predict(vectorizer.transform(["bitter terrorism optimism sober"])))

C:\Users\valik\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['anger'], dtype=object)

In [111]:
import eli5

In [120]:
eli5.show_weights(classifier, target_names=emotion_encoder.classes_, vec=vectorizer)

### Neural network

In [128]:
from keras.models import Sequential
from keras import layers

input_dim = X_train.shape[1]  # Number of features
output_dim = 4

model = Sequential()
model.add(layers.Dense(4, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(output_dim, activation='softmax'))


In [145]:
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 4)                 49000     
_________________________________________________________________
dense_8 (Dense)              (None, 4)                 20        
Total params: 49,020
Trainable params: 49,020
Non-trainable params: 0
_________________________________________________________________


In [146]:
history = model.fit(X_train, y_train,
                    epochs=100,
                    verbose=False,
                    validation_data=(X_test, y_test),
                    batch_size=10)

In [148]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.9867
Testing Accuracy:  0.7081



## Advanced neural network

In [186]:
#
from keras.preprocessing.text import Tokenizer

sentences_train = list(train.expression)
sentences_test = list(test.expression)

tokenizer = Tokenizer(num_words=8000)
tokenizer.fit_on_texts(sentences_train)

X_train = tokenizer.texts_to_sequences(sentences_train)
X_test = tokenizer.texts_to_sequences(sentences_test)

vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index

print(sentences_train[2])
print(X_train[2])

<subject> i asked for my parcel to be delivered to a pick up store not my address fuming poorcustomerservice
[1, 4, 380, 14, 11, 2474, 3, 16, 830, 3, 5, 610, 43, 1036, 22, 11, 1198, 259, 2475]


In [187]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 80

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

print(X_train[0, :])

[  56    2 2467 1797   65    2 2468 1429   11 2469  119    4 2470    2
 2471 1034  144  563    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0]


In [188]:
from keras.models import Sequential
from keras import layers

embedding_dim = 30

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.Flatten())
# model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 80, 30)            271710    
_________________________________________________________________
flatten_9 (Flatten)          (None, 2400)              0         
_________________________________________________________________
dense_34 (Dense)             (None, 4)                 9604      
Total params: 281,314
Trainable params: 281,314
Non-trainable params: 0
_________________________________________________________________


In [189]:
history = model.fit(X_train, y_train,
                    epochs=20,
                    validation_data=(X_test, y_test),
                    batch_size=10)

loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Train on 3613 samples, validate on 3142 samples
Epoch 1/20
3613/3613 [==============================] - 4s 1ms/step - loss: 1.3530 - acc: 0.3379 - val_loss: 1.3464 - val_acc: 0.3421
Epoch 2/20
3613/3613 [==============================] - 3s 795us/step - loss: 1.0269 - acc: 0.7371 - val_loss: 1.1651 - val_acc: 0.5598
Epoch 3/20
3613/3613 [==============================] - 3s 814us/step - loss: 0.4585 - acc: 0.9552 - val_loss: 1.0113 - val_acc: 0.6209
Epoch 4/20
3613/3613 [==============================] - 3s 844us/step - loss: 0.1953 - acc: 0.9762 - val_loss: 0.9589 - val_acc: 0.6461
Epoch 5/20
3613/3613 [==============================] - 3s 871us/step - loss: 0.1129 - acc: 0.9787 - val_loss: 0.9614 - val_acc: 0.6410
Epoch 6/20
3613/3613 [==============================] - 3s 882us/step - loss: 0.0811 - acc: 0.9776 - val_loss: 0.9594 - val_acc: 0.6426
Epoch 7/20
3550/3613 [============================>.] - ETA: 0s - loss: 0.0642 - acc: 0.9792

KeyboardInterrupt: 

In [203]:
from keras.models import Sequential
from keras import layers

embedding_dim = 100

model = Sequential()
model.add(layers.Embedding(input_dim=vocab_size, 
                           output_dim=embedding_dim, 
                           input_length=maxlen))
model.add(layers.GlobalMaxPool1D())
# model.add(layers.Dense(5, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4, activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_23 (Embedding)     (None, 80, 100)           905700    
_________________________________________________________________
global_max_pooling1d_14 (Glo (None, 100)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_41 (Dense)             (None, 4)                 404       
Total params: 906,104
Trainable params: 906,104
Non-trainable params: 0
_________________________________________________________________


In [204]:
history = model.fit(X_train, y_train,
                    epochs=50,
                    validation_data=(X_test, y_test),
                    batch_size=16)
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Train on 3613 samples, validate on 3142 samples
Epoch 1/50
3613/3613 [==============================] - 7s 2ms/step - loss: 1.3600 - acc: 0.3249 - val_loss: 1.3394 - val_acc: 0.3183
Epoch 2/50
3613/3613 [==============================] - 6s 2ms/step - loss: 1.2528 - acc: 0.4445 - val_loss: 1.2178 - val_acc: 0.5385
Epoch 3/50
3613/3613 [==============================] - 6s 2ms/step - loss: 1.0162 - acc: 0.7481 - val_loss: 0.9780 - val_acc: 0.7584
Epoch 4/50
3613/3613 [==============================] - 6s 2ms/step - loss: 0.7102 - acc: 0.8796 - val_loss: 0.7466 - val_acc: 0.8106
Epoch 5/50
3613/3613 [==============================] - 6s 2ms/step - loss: 0.4703 - acc: 0.9369 - val_loss: 0.6040 - val_acc: 0.8230
Epoch 6/50
3613/3613 [==============================] - 6s 2ms/step - loss: 0.3236 - acc: 0.9516 - val_loss: 0.5267 - val_acc: 0.8297
Epoch 7/50
3613/3613 [==============================] - 6s 2ms/step - loss: 0.2395 - acc: 0.9607 - val_loss: 0.4870 - val_acc: 0.8297
Epoch 8/50
361

KeyboardInterrupt: 